In [ ]:
!pip install  torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import RGCNConv
from torch_geometric.data import HeteroData

# 1. Define the Heterogeneous GNN Model
class HeteroGNN(nn.Module):
    def __init__(self, customer_dim, order_dim, item_dim, hidden_dim, num_relations):
        super(HeteroGNN, self).__init__()

        # Initial embedding layers for nodes with features
        self.customer_lin = nn.Linear(customer_dim, hidden_dim)
        self.order_lin = nn.Linear(order_dim, hidden_dim)
        self.item_emb = nn.Parameter(torch.randn(item_dim, hidden_dim))  # Learnable item embeddings

        # R-GCN layers for heterogeneous message passing (2 layers)
        self.conv1 = RGCNConv(hidden_dim, hidden_dim, num_relations=num_relations)
        self.conv2 = RGCNConv(hidden_dim, hidden_dim, num_relations=num_relations)

        # Link prediction scoring layer
        self.link_pred = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x_dict, edge_index_dict, edge_type):
        # Initial embeddings
        h_customer = F.relu(self.customer_lin(x_dict['customer']))
        h_order = F.relu(self.order_lin(x_dict['order']))
        h_item = self.item_emb  # Directly use learnable embeddings for items

        # Combine into a single node feature tensor
        h = torch.cat([h_customer, h_order, h_item], dim=0)

        # Heterogeneous GNN layers
        h = self.conv1(h, edge_index_dict['combined'], edge_type)
        h = F.relu(h)
        h = self.conv2(h, edge_index_dict['combined'], edge_type)

        # Split back into node types
        num_customers = x_dict['customer'].size(0)
        num_orders = x_dict['order'].size(0)
        num_items = x_dict['item'].size(0)

        h_customer = h[:num_customers]
        h_order = h[num_customers:num_customers + num_orders]
        h_item = h[num_customers + num_orders:]

        return {'customer': h_customer, 'order': h_order, 'item': h_item}

    def predict_link(self, h_order, h_item):
        # Predict Order -> Item edges
        combined = torch.cat([h_order, h_item], dim=-1)
        score = self.link_pred(combined)
        return score

# 2. Data Preparation
def create_hetero_data():
    data = HeteroData()

    # Dummy data (replace with your real data)
    num_customers, num_orders, num_items = 1, 10, 10

    # Node features
    data['customer'].x = torch.randn(num_customers, 4)  # [age, gender, income, location]
    data['order'].x = torch.randn(num_orders, 3)       # [status] (one-hot, e.g., 3 statuses)
    data['item'].x = torch.zeros(num_items, 1)         # Placeholder (no initial features)

    # Edge indices (example connectivity)
    data['customer', 'to', 'order'].edge_index = torch.tensor([
        [0, 1, 2,],  # Customer IDs
        [0, 1, 3,]   # Order IDs
    ], dtype=torch.long)

    data['order', 'to', 'item'].edge_index = torch.tensor([
        [0, 1, 2, ],  # Order IDs
        [0, 2, 5,]   # Item IDs
    ], dtype=torch.long)

    data['order', 'to', 'order'].edge_index = torch.tensor([
        [0, 1, 2,],  # Order IDs (source)
        [1, 2, 3,]   # Order IDs (target)
    ], dtype=torch.long)

    # Combine edges into a single edge_index and edge_type for R-GCN
    edge_index_list = [
        data['customer', 'to', 'order'].edge_index,
        data['order', 'to', 'item'].edge_index,
        data['order', 'to', 'order'].edge_index
    ]
    edge_type_list = [
        torch.zeros(edge_index_list[0].size(1), dtype=torch.long),  # Relation 0
        torch.ones(edge_index_list[1].size(1), dtype=torch.long),   # Relation 1
        torch.full((edge_index_list[2].size(1),), 2, dtype=torch.long)  # Relation 2
    ]

    data['combined'].edge_index = torch.cat(edge_index_list, dim=1)
    data['combined'].edge_type = torch.cat(edge_type_list)

    return data

# 3. Training Loop
def train(model, data, optimizer, num_epochs=100):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()

        # Forward pass
        h_dict = model(data.x_dict, data.edge_index_dict, data['combined'].edge_type)

        # Example: Predict Order -> Item edges (using training edges)
        edge_index_oi = data['order', 'to', 'item'].edge_index
        h_order = h_dict['order'][edge_index_oi[0]]
        h_item = h_dict['item'][edge_index_oi[1]]

        # Positive samples
        pos_scores = model.predict_link(h_order, h_item)

        # Negative sampling (random order-item pairs)
        neg_item_idx = torch.randint(0, data['item'].x.size(0), (edge_index_oi.size(1),))
        h_neg_item = h_dict['item'][neg_item_idx]
        neg_scores = model.predict_link(h_order, h_neg_item)

        # Loss (binary cross-entropy)
        scores = torch.cat([pos_scores, neg_scores])
        labels = torch.cat([torch.ones(pos_scores.size(0)), torch.zeros(neg_scores.size(0))])
        labels = labels.view(-1, 1)
        loss = F.binary_cross_entropy_with_logits(scores, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# 4. Inference: Recommend Items for a Given Order
def recommend_items(model, data, order_idx, top_k=5):
    model.eval()
    with torch.no_grad():
        h_dict = model(data.x_dict, data.edge_index_dict, data['combined'].edge_type)

        # Get embedding for the specific order
        h_order = h_dict['order'][order_idx].unsqueeze(0)  # Shape: [1, hidden_dim]

        # Score all items
        h_items = h_dict['item']  # Shape: [num_items, hidden_dim]
        h_order_expanded = h_order.expand_as(h_items)  # Broadcast to match item shape
        scores = model.predict_link(h_order_expanded, h_items).squeeze()

        # Get top-k items
        top_scores, top_items = scores.topk(top_k)
        return top_items, top_scores

# 5. Main Execution
def main():
    # Hyperparameters
    customer_dim = 4  # [age, gender, income, location]
    order_dim = 3     # [status] (one-hot)
    item_dim = 50     # Number of items
    hidden_dim = 64   # Embedding size
    num_relations = 3 # Customer->Order, Order->Item, Order->Order

    # Create data
    data = create_hetero_data()

    # Initialize model and optimizer
    model = HeteroGNN(customer_dim, order_dim, item_dim, hidden_dim, num_relations)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # Train the model
    train(model, data, optimizer)

    # Example: Recommend items for order with index 0
    top_items, top_scores = recommend_items(model, data, order_idx=0)
    print(f"Top recommended items: {top_items.tolist()}")
    print(f"Scores: {top_scores.tolist()}")

if __name__ == "__main__":
    main()

Epoch 0, Loss: 0.7273
Epoch 10, Loss: 0.0073
Epoch 20, Loss: 0.0244
Epoch 30, Loss: 0.0173
Epoch 40, Loss: 0.0826
Epoch 50, Loss: 0.0068
Epoch 60, Loss: 0.0261
Epoch 70, Loss: 0.0211
Epoch 80, Loss: 0.1415
Epoch 90, Loss: 0.0210
Top recommended items: [0, 2, 5, 27, 6]
Scores: [8.746610641479492, 6.771003723144531, 6.005368709564209, -3.439739227294922, -6.8245697021484375]


In [ ]:
create_hetero_data()

HeteroData(
  customer={ x=[100, 4] },
  order={ x=[200, 3] },
  item={ x=[50, 1] },
  combined={
    edge_index=[2, 9],
    edge_type=[9],
  },
  (customer, to, order)={ edge_index=[2, 3] },
  (order, to, item)={ edge_index=[2, 3] },
  (order, to, order)={ edge_index=[2, 3] }
)